In [53]:
# %% [markdown]
# Retrieval system using HOG descriptors + mAP@1 and mAP@5
# This notebook:
# 1. Builds (or loads) HOG descriptors for the database (BBDD)
# 2. Extracts HOG for the query images
# 3. Retrieves the most similar database images using cosine similarity
# 4. Evaluates using mAP@1 and mAP@5 based on gt_corresps.pkl

# %%
import os
import cv2
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from noise_filter import preprocess_image
import background_remover_w2 as background_remover
from image_split import split_images
import matplotlib.pyplot as plt

# --- paths (adjust to your structure) ---
QUERY_FOLDER = "../Data/Week4/qsd1_w4/"     # query images
DB_FOLDER    = "../Data/BBDD/"        # database (gallery) images
GT_PATH      = "../Data/Week4/qsd1_w4/gt_corresps.pkl"

# where to store precomputed DB descriptors
DESC_DB_PATH = "results/descriptors_db_hog.pkl"

os.makedirs("results", exist_ok=True)


In [54]:
# %%
def compute_hog_descriptor(
    img_bgr,
    size=(256, 256),
    cell=(8, 8),
    block=(16, 16),
    block_stride=(8, 8),
    nbins=9,
):
    """
    Compute a HOG descriptor for a BGR image.
    We:
    1. convert to grayscale
    2. resize to a fixed size so all descriptors have the same length
    3. compute HOG
    4. L2-normalize the final vector
    """
    # 1) BGR -> Gray
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    # 2) fixed resize to make descriptor length stable
    gray = cv2.resize(gray, size)

    # 3) create HOG object with consistent parameters
    hog = cv2.HOGDescriptor(
        _winSize=size,
        _blockSize=block,
        _blockStride=block_stride,
        _cellSize=cell,
        _nbins=nbins,
    )
    desc = hog.compute(gray)  # shape (N, 1)
    desc = desc.flatten().astype(np.float32)

    # 4) L2 normalization (good for cosine and distance-based retrieval)
    norm = np.linalg.norm(desc)
    if norm > 0:
        desc = desc / norm

    return desc


def load_images_from_folder(folder):
    """
    Loads all .jpg/ images from a folder, sorted by filename.
    Returns:
        names: list of filenames
        imgs:  list of loaded cv2 images
    """
    names = sorted(
        [
            f
            for f in os.listdir(folder)
            if f.lower().endswith((".jpg"))
        ]
    )
    imgs = []
    for name in names:
        path = os.path.join(folder, name)
        img = cv2.imread(path)
        if img is None:
            print(f"⚠️ Could not read {path}")
            continue
        imgs.append(img)
    return names, imgs


def precision_at_k(retrieved_indices, gt_indices, k):
    """
    Compute precision@k for one query.

    retrieved_indices: list of database indices sorted by similarity (best first)
    gt_indices: list of correct database indices for that query (can be 1 or more)
    k: cutoff (1, 5, ...)

    precision@k = (# of retrieved items in top-k that are in GT) / k
    """
    retrieved_k = retrieved_indices[:k]
    hits = sum(1 for r in retrieved_k if r in gt_indices)
    return hits / k


def mean_average_precision_at_k(all_retrieved, all_gts, k):
    """
    Compute mean precision@k over all queries.
    (Many assignments call this mAP@k when GT is small.)
    """
    assert len(all_retrieved) == len(all_gts)
    precisions = []
    for retrieved, gts in zip(all_retrieved, all_gts):
        p = precision_at_k(retrieved, gts, k)
        precisions.append(p)
    return np.mean(precisions)


In [55]:
# %% 
# 1. Build or load database descriptors (HOG)

if os.path.exists(DESC_DB_PATH):
    print(f"✅ Loading DB descriptors from {DESC_DB_PATH}")
    with open(DESC_DB_PATH, "rb") as f:
        data = pickle.load(f)
        db_descs = data["desc_gt"]    # list of numpy arrays
        db_names = data["gt_names"]   # list of image filenames
else:
    print("🧠 Computing HOG descriptors for database images...")
    db_names, db_imgs = load_images_from_folder(DB_FOLDER)
    db_descs = []
    for img, name in zip(db_imgs, db_names):
        print("Processing image",name)
        d = compute_hog_descriptor(img)
        db_descs.append(d)

    # store only numpy arrays (serializable) → no cv2.KeyPoint here
    with open(DESC_DB_PATH, "wb") as f:
        pickle.dump({"desc_gt": db_descs, "gt_names": db_names}, f)
    print(f"✅ Saved DB descriptors to {DESC_DB_PATH}")


✅ Loading DB descriptors from results/descriptors_db_hog.pkl


In [56]:
# %%
# 2. Load query images and ground-truth correspondences

print("📥 Loading query images...")
q_names, q_imgs = load_images_from_folder(QUERY_FOLDER)

print("📥 Loading ground-truth correspondences...")
with open(GT_PATH, "rb") as f:
    gt_corresps = pickle.load(f)   # e.g. [[236], [107], [280, 285], ...]
print(f"→ {len(gt_corresps)} GT entries loaded")


📥 Loading query images...
📥 Loading ground-truth correspondences...
→ 30 GT entries loaded


In [57]:
# %%
# 3. Compute HOG descriptors for queries

print("🧠 Computing HOG for queries...")
desc_query = []
for img, img_name in zip(q_imgs, q_names):
    print("Processing image",img_name)
    # Split possible multiple artworks
    splitted = split_images(img)

    if splitted[0] is True:
        splitted = splitted[1]  # two artworks detected
        left_artwork, right_artwork = splitted

        left_artwork = preprocess_image(left_artwork)
        right_artwork = preprocess_image(right_artwork)

        iml, left_mask, left_output, _ = background_remover.remove_background_morphological_gradient(left_artwork)
        imr, right_mask, right_output, _ = background_remover.remove_background_morphological_gradient(right_artwork)

        # Crop each artwork to its mask bounding box (no black borders)
        left_cropped = background_remover.crop_to_mask_rectangle(left_artwork, left_mask)
        right_cropped = background_remover.crop_to_mask_rectangle(right_artwork, right_mask)

        # Extract descriptors
        #kps_left = detect_keypoints(left_cropped, METHOD)
        #kps_right = detect_keypoints(right_cropped, METHOD)
        
        desc_left  = compute_hog_descriptor(left_cropped)
        desc_right = compute_hog_descriptor(right_cropped)

        desc_query.append([desc_left, desc_right])
        
        """plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(left_cropped, cv2.COLOR_BGR2RGB))
        plt.title(f"Left Artwork: {img_name}")
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(right_cropped, cv2.COLOR_BGR2RGB))
        plt.title(f"Right Artwork: {img_name}")
        plt.axis('off')
        
        plt.show()"""

    else:  # single artwork
        splitted = splitted[1]  # single artwork
        img = preprocess_image(splitted)
        im, mask, output, _ = background_remover.remove_background_morphological_gradient(img)

        # Crop to mask bounding box (remove black)
        cropped = background_remover.crop_to_mask_rectangle(img, mask)

        # Extract descriptor
        #kps = detect_keypoints(cropped, METHOD)
        desc = compute_hog_descriptor(cropped)
            
        desc_query.append([desc])  # keep structure consistent
        
        """plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
        plt.title(f"{img_name}")
        plt.axis('off')
        plt.show()"""

# stack DB descriptors into a matrix so cosine_similarity is fast
db_mat = np.vstack(db_descs)   # shape (N_db, D)


🧠 Computing HOG for queries...
Processing image 00000.jpg
Processing image 00001.jpg
Processing image 00002.jpg
Processing image 00003.jpg
Processing image 00004.jpg
Processing image 00005.jpg
Processing image 00006.jpg
Processing image 00007.jpg
Processing image 00008.jpg
Processing image 00009.jpg
Processing image 00010.jpg
Processing image 00011.jpg
Processing image 00012.jpg
Processing image 00013.jpg
Processing image 00014.jpg
Processing image 00015.jpg
Processing image 00016.jpg
Processing image 00017.jpg
Processing image 00018.jpg
Processing image 00019.jpg
Processing image 00020.jpg
Processing image 00021.jpg
Processing image 00022.jpg
Processing image 00023.jpg
Processing image 00024.jpg
Processing image 00025.jpg
Processing image 00026.jpg
Processing image 00027.jpg
Processing image 00028.jpg
Processing image 00029.jpg


In [58]:
# --- 4. Compute mAP@1 and mAP@5 ---
def compute_map_at_k(desc_query, desc_gt, gt_corresps, k=5):
    """
    Compute mean Average Precision at K.
    If a query image has multiple artworks, we pair each sub-descriptor with its matching GT index
    (i.e., descs[j] -> gt_corresps[i][j]) when lengths match.
    """
    aps = []

    for i, descs in enumerate(desc_query):
        q_gt = gt_corresps[i]
        # Ensure list
        if not isinstance(q_gt, list):
            q_gt = [q_gt]

        for desc in descs:
            sims = cosine_similarity([desc], desc_gt)[0]
            ranked_indices = np.argsort(-sims)[:k]

            num_relevant = len(q_gt)
            num_correct = 0
            precision_at_i = []

            for rank, idx in enumerate(ranked_indices, start=1):
                if idx in q_gt:
                    num_correct += 1
                    precision_at_i.append(num_correct / rank)

            ap = np.sum(precision_at_i) / num_relevant if num_relevant > 0 else 0
            aps.append(ap)

    return float(np.mean(aps)) if aps else 0.0



map1 = compute_map_at_k(desc_query, db_descs, gt_corresps, k=1)
map5 = compute_map_at_k(desc_query, db_descs, gt_corresps, k=5)

print(f"\n✅ mAP@1 = {map1:.4f}")
print(f"✅ mAP@5 = {map5:.4f}")



✅ mAP@1 = 0.3857
✅ mAP@5 = 0.4000
